In [2]:
import urllib.request as request
from bs4 import BeautifulSoup
import sqlite3
import re
from datetime import datetime
import ipdb;ipdb.set_trace()

startTime = datetime.now()

# http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks-in-python
def chunks(l, n):
    """ Yield successive n-sized chunks from l.
    """
    for i in range(0, len(l), n):
        yield l[i:i+n]

base_url = 'http://espn.go.com'

teams_url = 'http://espn.go.com/nba/teams'
html_teams = request.urlopen(teams_url)

soup_teams = BeautifulSoup(html_teams,'lxml')
urls = soup_teams.find_all(href=re.compile('/nba/teams/stats'))

team_urls = [base_url+url['href'] for url in urls]

for team in team_urls:
    html_team = request.urlopen(team)
    soup_team = BeautifulSoup(html_team,'lxml')
    html_rows = soup_team.find_all('tr', class_=re.compile('player'))
    
    player_urls = [row.a['href'].replace('_','stats/_') for row in html_rows]
    
    for player in player_urls:
        player_id   = player.split('/')[8]
        html_player = request.urlopen(player)
        soup_player = BeautifulSoup(html_player,'lxml')
        
        soup_name = soup_player.find('meta', property='og:title')
        player_name = soup_name['content']
        
        regular_season_stats = soup_player.find_all('tr', class_=re.compile('row'))
        
        size = int(len(regular_season_stats)/3)
        
        season_avgs_slice        = slice(0,size)
        #season_totals_slice      = slice(size,size*2)
        #season_misc_totals_slice = slice(size*2,size*3)
        
        regular_season_avgs = regular_season_stats[season_avgs_slice]
        #regular_season_totals = regular_season_stats[season_totals_slice]
        #regular_season_misc_totals = regular_season_stats[season_misc_totals_slice]
        
        avgs = []
        for row in regular_season_avgs:
            if len(row) == 20:  # Only accept row that contains complete data
                for data in row:
                    avgs.append(data.get_text())
            else:
                pass
                
        index = 0 # insert the player ID before the player's season
        increment = 0
        for row in range(len(regular_season_avgs)):
            avgs.insert(index + increment, player_id)
            index = index + 20  # There are 20 columns in the season avgs section
            increment = increment + 1
            
        index = 1 # insert the player's name after the player's ID
        increment = 0
        for row in range(len(regular_season_avgs)):
            avgs.insert(index + increment, player_name)
            index = index + 21  # There are 21 columns in the season avgs section since I've just added player ID
            increment = increment + 1

        conn = sqlite3.connect('/home/pybokeh/databases/nba.db')
        c = conn.cursor()

        for data in chunks(avgs,22):
            try:
                c.execute('INSERT INTO regular_season_avgs VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', data)
            except:
                pass
            conn.commit()
        conn.close()
        
print(datetime.now() - startTime)

MultipleInstanceError: Multiple incompatible subclass instances of TerminalIPythonApp are being created.